# Imports

In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.io as spio
import matplotlib.pyplot as plt
from matplotlib import colors

from pymor.models.iosys import LTIModel
from pymor.operators.constructions import LincombOperator
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.parameters.functionals import ProjectionParameterFunctional

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

# Build model

In [ ]:
mat = spio.loadmat('../lectures/data/ABCE.mat')
mat.keys()

In [ ]:
A0 = NumpyMatrixOperator(mat['A0'])
Amu = NumpyMatrixOperator(sum(0.2 * i * mat[f'A{i}'] for i in range(1, 5)))
A = LincombOperator((A0, Amu), (1, ProjectionParameterFunctional('mu', index=0)))
B = NumpyMatrixOperator(mat['B'])
C = NumpyMatrixOperator(mat['C'])
E = NumpyMatrixOperator(mat['E'])

In [ ]:
fom = LTIModel(A, B, C, E=E)

In [ ]:
fom

In [ ]:
print(fom)

# Parametric magnitude plot

In [ ]:
w_list = np.logspace(-4, 4, 10)
mu_list = np.logspace(-6, 2, 10)
tf = np.array([[spla.norm(...)  # replace "..." with code to evaluate H(i * w, mu)
                for w in w_list]
               for mu in mu_list])

In [ ]:
fig, ax = plt.subplots()
pcm = ax.pcolormesh(w_list, mu_list, tf, shading='gouraud',
                    norm=colors.LogNorm(vmin=tf.min(), vmax=tf.max()))
ax.set_xscale('log')
ax.set_yscale('log')
_ = fig.colorbar(pcm)

# Hankel singular values

In [ ]:
mu_list_hsv = np.logspace(-6, 2, 3)
hsv_list = [... for mu in mu_list_hsv]  # write code to get Hankel singular values for parameter mu

In [ ]:
fig, ax = plt.subplots()
for mu, hsv in zip(mu_list_hsv, hsv_list):
    ax.semilogy(hsv, '.-', label=f'mu = {mu}')
_ = ax.legend()

# Balanced truncation

Reduce the model for $\mu = 0.01$ using balanced truncation to order $10$.

In [ ]:
from pymor.reductors.bt import BTReductor

In [ ]:
bt = BTReductor(...)

In [ ]:
rom0 = bt.reduce(...)

Is the reduced model parametric?

In [ ]:
rom0

Use basis matrices from balanced truncation to project the model.

In [ ]:
from pymor.reductors.basic import LTIPGReductor

In [ ]:
pg = LTIPGReductor(fom, ...)  # use bt.V and bt.W

In [ ]:
rom = pg.reduce()

In [ ]:
rom

In [ ]:
err = fom - rom

In [ ]:
w_list = np.logspace(-4, 4, 10)
mu_list = np.logspace(-6, 2, 10)
tf_err = np.array([[spla.norm(...)
                    for w in w_list]
                   for mu in mu_list])

In [ ]:
fig, ax = plt.subplots()
pcm = ax.pcolormesh(w_list, mu_list, tf_err, shading='gouraud',
                    norm=colors.LogNorm(vmin=tf_err.min(), vmax=tf_err.max()))
ax.set_xscale('log')
ax.set_yscale('log')
_ = fig.colorbar(pcm)